In [1]:
import pandas as pd

df = pd.read_csv('./filename.csv')
df = df.drop(['Bid Price','Bid Size','Ask Price','Ask Size','Side','Unnamed: 0'],axis =1 )
df

,Date-Time,Price,Volume,BuyV,SellV
0,2022-12-27 16:23:20.066388299,0.04,1,0,1
1,2022-12-27 16:24:13.735070427,0.03,1,0,1
2,2023-01-10 19:18:31.819747797,0.09,3,0,3
3,2023-01-10 19:18:31.827713125,0.09,1,0,1
4,2023-01-10 19:18:31.827713125,0.09,1,0,1
...,...,...,...,...,...
36765,2023-12-28 15:08:18.724964834,-0.10,1,0,1
36766,2024-01-03 12:58:52.359634381,-0.13,1,0,1
36767,2024-01-03 12:58:53.659706131,-0.13,1,0,1
36768,2024-01-04 19:11:54.480149223,0.16,1,1,0


In [2]:
df['Date-Time'] = pd.to_datetime(df['Date-Time'])
df = df.sort_values(by='Date-Time')
df.set_index('Date-Time',inplace=True)
# df.to_csv('./test.csv')

In [3]:
resampled_df = df.resample('T').agg({
    'Price': ['first', 'max', 'min', 'last'],
    'Volume': 'sum',
    'BuyV': 'sum',
    'SellV': 'sum'
})

# Flatten the MultiIndex columns
resampled_df.columns = ['_'.join(col).strip() for col in resampled_df.columns.values]

# Rename columns for clarity
resampled_df.rename(columns={
    'Price_first': 'Open', 
    'Price_max': 'High', 
    'Price_min': 'Low', 
    'Price_last': 'Close'
}, inplace=True)

# Reset index to have Date-Time as a column again
resampled_df.reset_index(inplace=True)

# Display the resulting DataFrame
resampled_df = resampled_df.dropna()

C:\Users\anmol.chopra\AppData\Local\Temp\ipykernel_31256\433845259.py:1: FutureWarning: 'T' is deprecated and will be removed in a future version, please use 'min' instead.
  resampled_df = df.resample('T').agg({


In [4]:
# resampled_df.to_csv('./test.csv',index=False)

In [5]:
def is_dst(date):
    if date.tzinfo is None:
        date = date.tz_localize('UTC').tz_convert('America/New_York')  # Adjust timezone as per your requirement
    return date.dst() != pd.Timedelta(0)

# Create the DST and Non_DST columns
resampled_df['DST'] = resampled_df['Date-Time'].apply(lambda x: 1 if is_dst(pd.Timestamp(x)) else 0)

resampled_df

,Date-Time,Open,High,Low,Close,Volume_sum,BuyV_sum,SellV_sum,DST
0,2022-12-27 16:23:00,0.04,0.04,0.04,0.04,2,0,2,0
1,2022-12-27 16:24:00,0.03,0.03,0.03,0.03,2,0,2,0
20335,2023-01-10 19:18:00,0.09,0.09,0.05,0.06,84,40,44,0
21415,2023-01-11 13:18:00,0.07,0.07,0.07,0.07,20,0,20,0
21508,2023-01-11 14:51:00,0.11,0.11,0.11,0.11,4,4,0,0
...,...,...,...,...,...,...,...,...,...
525771,2023-12-27 19:14:00,-0.06,0.02,-0.06,0.02,28,28,0,0
525773,2023-12-27 19:16:00,-0.01,-0.01,-0.01,-0.01,2,2,0,0
526965,2023-12-28 15:08:00,-0.10,-0.10,-0.10,-0.10,2,0,2,0
535475,2024-01-03 12:58:00,-0.13,-0.13,-0.13,-0.13,4,0,4,0


In [11]:
# resampled_df.to_csv('./test.csv',index=False)

In [7]:
resampled_df['Time'] = resampled_df['Date-Time'].dt.time

times_to_filter = [pd.to_datetime('18:14').time(), pd.to_datetime('19:14').time()]

# Filter the dataframe
filtered_df = resampled_df[resampled_df['Time'].isin(times_to_filter)]
filtered_df = filtered_df.drop(['Time'],axis =1)
filtered_df

,Date-Time,Open,High,Low,Close,Volume_sum,BuyV_sum,SellV_sum,DST
21771,2023-01-11 19:14:00,0.09,0.09,0.09,0.09,2,0,2,0
53451,2023-02-02 19:14:00,0.12,0.12,0.12,0.12,14,14,0,0
64971,2023-02-10 19:14:00,0.13,0.13,0.13,0.13,6,6,0,0
70731,2023-02-14 19:14:00,0.14,0.14,0.14,0.14,6,6,0,0
90891,2023-02-28 19:14:00,0.15,0.15,0.15,0.15,8,8,0,0
...,...,...,...,...,...,...,...,...,...
512811,2023-12-18 19:14:00,0.14,0.14,0.13,0.13,12,2,10,0
514251,2023-12-19 19:14:00,0.10,0.12,0.10,0.12,20,10,10,0
515691,2023-12-20 19:14:00,0.02,0.02,0.02,0.02,2,2,0,0
517071,2023-12-21 18:14:00,0.01,0.01,0.00,0.00,4,2,2,0


In [8]:
condition_1 = (filtered_df['Date-Time'].dt.time == pd.to_datetime('18:14').time()) & (filtered_df['DST'] == 1)
condition_2 = (filtered_df['Date-Time'].dt.time == pd.to_datetime('19:14').time()) & (filtered_df['DST'] == 0)

# Filter the DataFrame based on the conditions
filtered_df = filtered_df[condition_1 | condition_2]

In [10]:
filtered_df.to_csv('./test.csv',index=False)

In [16]:
import pandas as pd

In [19]:
df1 = pd.read_csv('Data.csv')
df2 = pd.read_excel('Peru.xlsx',sheet_name=1)

In [24]:
# df1 = df1.drop(['HG.OPEN','HG.HIGH','HG.LOW'],axis=1)
df1 = df1.rename(columns={'Timestamp': 'Date'})
df1

,Date,HG.CLOSE
0,2012-09-10,4.0820
1,2012-09-11,4.0905
2,2012-09-12,4.0860
3,2012-09-13,4.1035
4,2012-09-14,4.2260
...,...,...
2994,2024-08-01,4.0825
2995,2024-08-02,4.1030
2996,2024-08-05,3.9990
2997,2024-08-06,4.0270


In [25]:
df1['Date'] = pd.to_datetime(df1['Date'])
df2['Date'] = pd.to_datetime(df2['Date'])

# Merge dataframes on the Date column
merged_df = pd.merge(df1, df2, on='Date', how='inner')
merged_df.to_csv('data-peru2.csv')